In [134]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import feature_extraction, model_selection, naive_bayes, metrics, svm
from IPython.display import Image
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, GridSearchCV
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import string
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline  

## Dataset
Pandas provide a .read_csv() function. the paramter engine='python' was needed here.
We can see in the output that there are two columns "text" and "spam" 

In [129]:
data = pd.read_csv('/home/shreyansh/Downloads/emails.csv', encoding='latin-1', engine='python')
data.head(n=10)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
5,"Subject: great nnews hello , welcome to medzo...",1
6,Subject: here ' s a hot play in motion homela...,1
7,Subject: save your money buy getting this thin...,1
8,Subject: undeliverable : home based business f...,1
9,Subject: save your money buy getting this thin...,1


data['text']=str(data['text']).translate(str.maketrans("","", string.punctuation))
data.head(n=10)

In [137]:
def stemmer (text):
    text = text.split()
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [138]:
data['text']=data['text'].apply(stemmer)
data.head(n=10)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trade gunslinger fanny be m...,1
2,Subject: unbelievable new home make easy im wa...,1
3,Subject: 4 color print special request additio...,1
4,"Subject: do not have money , get software cd f...",1
5,"Subject: great nnews hello , welcome to medzon...",1
6,Subject: here ' s a hot play in motion homelan...,1
7,Subject: save your money buy get this thing he...,1
8,Subject: undeliverable : home base business fo...,1
9,Subject: save your money buy get this thing he...,1


In [139]:
f = TfidfVectorizer(stop_words = 'english')
X = f.fit_transform(data["text"])
print(np.shape(X))
print(X)

(5728, 31496)
  (0, 27319)	0.01789925274651154
  (0, 20003)	0.12315952355110002
  (0, 16005)	0.1439790538781215
  (0, 8839)	0.07461302327675667
  (0, 15083)	0.09264580657899109
  (0, 18057)	0.11314281912190569
  (0, 23878)	0.0685057414293913
  (0, 14203)	0.0788689798179666
  (0, 23974)	0.11446929644601661
  (0, 8277)	0.1588773946197651
  (0, 18514)	0.20146108972371513
  (0, 27516)	0.1243147148528275
  (0, 15539)	0.04519203921838758
  (0, 16049)	0.13556634012647437
  (0, 13705)	0.10318943732128652
  (0, 7259)	0.11517131790364561
  (0, 17897)	0.346609143251911
  (0, 27300)	0.1243147148528275
  (0, 27030)	0.12315952355110002
  (0, 21287)	0.09244355298357776
  (0, 30418)	0.13368602555037692
  (0, 18323)	0.12747726658520708
  (0, 27877)	0.09264580657899109
  (0, 10904)	0.0932666521480701
  (0, 23145)	0.17187814764995407
  :	:
  (5727, 11202)	0.04039312371667492
  (5727, 9080)	0.06105114679195972
  (5727, 11026)	0.04443943586899393
  (5727, 26923)	0.05556064332158135
  (5727, 10725)	0.055560

In [140]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, data['spam'], test_size=0.30, random_state=42)


In [141]:
list_C = np.arange(800,900,100)
list_gamma = np.arange(0.001,0.006,0.001)
parameters = [{'gamma': list_gamma,
                     'C': list_C}]
print(list_C)
print(list_gamma)
print(parameters)

[800]
[0.001 0.002 0.003 0.004 0.005]
[{'gamma': array([0.001, 0.002, 0.003, 0.004, 0.005]), 'C': array([800])}]


In [142]:
svc = svm.SVC(kernel='rbf')
gridsearch = GridSearchCV(svc, parameters, cv=2, scoring='f1', n_jobs=-1)
gridsearch.fit(X_train,y_train)
gridsearch.best_params_

{'C': 800, 'gamma': 0.001}

In [143]:
gridsearch.best_score_

0.9786979210869811

In [144]:
m_confusion_test = metrics.confusion_matrix(y_test, gridsearch.best_estimator_.predict(X_test))
pd.DataFrame(data = m_confusion_test, columns = ['Predicted 0', 'Predicted 1'],
            index = ['Actual 0', 'Actual 1'])

,Predicted 0,Predicted 1
Actual 0,1274,4
Actual 1,13,428
